<a href="https://colab.research.google.com/github/tuky-g/LLM-QAproject/blob/main/2_QA_chat_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [10]:
!pip install jupyter_bokeh
!pip install langchain_community
!pip install -qU chromadb langchain-chroma
!pip install tiktoken
!pip install --upgrade --quiet  langchain sentence_transformers
!pip install langchain_huggingface


In [11]:
# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
# from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [12]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

from chromadb.config import Settings

In [13]:
import panel as pn
import param

In [14]:
import os
import sys
sys.path.append('../..')
import panel as pn
import param

# import panel as pn  # GUI
# pn.extension()


In [15]:
HF_API_KEY = os.environ.get('HF_TOKEN')

# conect to drive

In [16]:
#connect to folder in drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# retrival

In [29]:
persist_directory='/content/drive/MyDrive/LLM-QAproject/docs/chroma2/'
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"
                                  # ,clean_up_tokenization_spaces=True  # Add this line
                                  )


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
# load from disk
vectordb = Chroma(
    collection_name="foo",
    persist_directory=persist_directory,
    embedding_function=embeddings
)

vectordb._collection


Collection(id=60d2495f-3da9-4c02-a411-4c51ca0f12d6, name=foo)

In [33]:
print(vectordb._collection.count())

443


In [34]:
question = "What is a NetBEUI?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

# chat

In [35]:
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


# Initialize your retriever
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 4}) # Your retriever

# Initialize the language model
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

# Define the prompt to contextualize the question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Define the prompt to answer the question
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create the question-answering chain
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create the retrieval chain
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain
)

In [36]:
# Usage example
chat_history = []  # Collect chat history here (a sequence of messages)
query = "What is a NetBEUI?"
response = rag_chain.invoke({"input": query, "chat_history": chat_history})
print(response)

{'input': 'What is a NetBEUI?', 'chat_history': [], 'context': [Document(metadata={'page': 4, 'source': '/content/drive/MyDrive/LLM-QAproject/pdf_vlite/Copy of 01. Advanced Penetration Testing.pdf'}, page_content='Figure 1.9 Blank document carrying macro payload.\nFigure 1.10 A little more convincing.\nFigure 1.11 Initial basic Command and Control infrastructure.\nFigure 1.12 The completed attack with complete access to the medical\nrecords.\nChapter 2: Stealing Research\nFigure 2.1 Permit all local Java code to run in the browser.\nFigure 2.2 Java applet running in the browser.\nFigure 2.3 The upgraded framework handles multiple hosts and\noperating systems.\nChapter 3: Twenty-First Century Heist\nFigure 3.1 The beauty of this setup is that if your C2 is disrupted by\nsecurity operations, you can point your DNS at another server.\nFigure 3.2 A basic intrusion monitoring setup.\nFigure 3.3 Mmmmmm. Stealthy.\nChapter 4: Pharma Karma\nFigure 4.1 This image from cvedetails shows 56 code e

In [44]:
print(response['context'] [1])

page_content='Payload Delivery Part VIII: Miscellaneous Rich Web
Content
We've talked about Java applets and touched on Adobe Flash as attack
vectors. However, as Oracle has expressed a desire to replace applets in their
current form and as the browser makers have lost all patience with Adobe
over their complete lack of secure coding practices, neither of these
technologies are going to be around forever. Their successors are already in
active deployment and are suitable for use in APT modeling attacks. Although
they are very different from each other technologically, the way they offer
content to the user is (visually) not all that dissimilar, so it makes sense to
talk about the two together.
Java Web Start
JWS applications don't run inside the browser but are generally deployed
through the browser interface. From a software development perspective, this
has several advantages, but mainly it allows much more refined memory
management and 
indeed the allocation of much more memory than

# chat

In [38]:


def load_db(k):
    persist_directory='/content/drive/MyDrive/LLM-QAproject/docs/chroma4/'
    # define embedding
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"
                                  # ,clean_up_tokenization_spaces=True  # Add this line
                                  )
    # load db
    vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)
    # define retriever
    retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chatbot chain. Memory is managed externally.
    # model
    llm = HuggingFaceEndpoint(
        repo_id="HuggingFaceH4/zephyr-7b-beta",
        task="text-generation",
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.03,
    )
    chat_model = ChatHuggingFace(llm=llm)
    #memory
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    # qa
    qa = ConversationalRetrievalChain.from_llm(
        chat_model,
        retriever=retriever,
        memory=memory
    )
    return qa

In [39]:


class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.qa = load_db(4)


    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["question"]
        self.answer = result['answer']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return


In [40]:
import panel as pn  # GUI
pn.extension()  # Load Panel extension

In [41]:
cb = cbfs()


inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp)


tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

<ipython-input-38-f112ff7a6b5d>:22: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()

<ipython-input-39-d87473d7a798>:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self.qa({"question": query, "chat_history": self.chat_history})
